## Importing the libraries

In [1]:
# Libraries for visualization
import plotly as py
from plotly.offline import plot, iplot
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

# Libraries for data reading and handling
from datetime import datetime
import pandas
import numpy as np

# Libraries for neural net trainingand evaluation
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import math

In [2]:
# Checking the tensorflow version
tf.__version__

'1.13.1'

## Functions for reading in the data and removing the outliers

In [3]:
"""
Reads in the 50 scenes in to the memory and then slices the data frame to achieve optimization
Takes 1.72 sec to read the 50 scenes file into memory
Slicing each scene takes 0.32 milli seconds on average
"""
def read_file(file_path):

    giant_df = pandas.read_csv(file_path, usecols=[1,2,3,4], names=["lz","X","Y","Z"])
    num_of_scenes = len(giant_df)/72000
    dataframes = []

    for i in range(int(num_of_scenes)):
        start = i*72000
        end = start+72000
        dataframes.append(giant_df.iloc[start:end,:])

    return dataframes

In [4]:
"""
Takes 0.36 sec to remove the outliers in each scene
"""

# Function to remove outliers
def remove_outliers(dataframes, number_of_scenes, path_to_pkl):
    object_points = []
    outliers = pandas.read_pickle(path_to_pkl)
    max_rad = []
    min_rad = []
    for i in range(64):
        max_rad.append(outliers[outliers['lz']==i]['max'].tolist()[0])
        min_rad.append(outliers[outliers['lz']==i]['min'].tolist()[0])

    for i in range(number_of_scenes):
        df = dataframes[i]
        df['radius'] = df.X.pow(2).add(df.Y.pow(2).add(df.Z.pow(2))).pow(0.5).round(1)
        df.drop(df[df['radius']==0].index, inplace=True)
        temp_out = pandas.DataFrame()
        for j in range(64):
            dummy_df = df[df['lz']==j]
            bool_vec = ~((dummy_df['radius']<=max_rad[j]) & (dummy_df['radius']>=min_rad[j]))
            temp_out = temp_out.append(dummy_df[bool_vec])
        object_points.append(temp_out)

    return object_points

## Function for visualizing the data

In [5]:
def visualize(object_data_frame, view):
    x = tuple(object_data_frame['X'].tolist())
    y = tuple(object_data_frame['Y'].tolist())
    z = tuple(object_data_frame['Z'].tolist())
    radius = tuple(object_data_frame['radius'].tolist())
    if view==1:
        x_points = x
        y_points = z
    elif view==2:
        x_points = x
        y_points = y
    elif view==3:
        x_points = z
        y_points = y
    else:
        print("Not a valid view")
        x_points = x
        y_points = z
    trace = go.Scatter(
            x = x_points,
            y = y_points,
            mode = 'markers',
            marker = dict(
                        color=radius,
                        size = 2,
                        opacity = 1)
            )
    layout = go.Layout(
                    scene = dict(
                    
                    xaxis = dict(
                        range = [-5,30]
                        ),
                    yaxis = dict(
                        range = [-120,120]
                        )
                    )
                )
    
    data = [trace]
    fig = go.Figure(data=data,layout = layout)
    iplot(fig)

## Path variables and objects

In [6]:
# objects
object_names = {0: 'Atm', 1: 'Bench', 2: 'BigSassafras', 3: 'BmwX5Simple', \
                4: 'ClothRecyclingContainer', 5: 'Cypress', 6: 'DrinkingFountain',\
                7: 'ElectricalCabinet', 8: 'EmergencyPhone', 9: 'FireHydrant',\
                10: 'GlassRecyclingContainer', 11: 'IceFreezerContainer', 12: 'Mailbox',\
                13: 'MetallicTrash', 14: 'MotorbikeSimple', 15: 'Oak', 16: 'OldBench',\
                17: 'Pedestrian', 18: 'PhoneBooth', 19: 'PublicBin', 20: 'Sassafras',\
                21: 'ScooterSimple', 22: 'set1', 23: 'ToyotaPriusSimple', 24: 'Tractor',\
                25: 'TrashBin', 26: 'TrashContainer', 27: 'UndergroundContainer',\
                28: 'WorkTrashContainer'}

In [7]:
# Select the objects from above
user_choice = 22

# Path to dataset in file
folder_path = "/home/samba693/DataChallenge/debs2019_initial_dataset"
file_path = "{}/{}/in.csv".format(folder_path, object_names[user_choice])

# number of scenes to read in must be less than or equal to 50
num_of_scenes = 50

# Path to the outliers pkl file
outliers_path = "../object-net/outliers.pkl"

## Function to visualize desired number of scenes in animation

In [8]:
#### Not so import to look at #####
"""
This function creates the visualization of the object points in particular view for given num_of_scenes

"""
def visualize_50_html(num_of_scenes, view,object_name = object_names[user_choice]):

    # Creating the list of scenes
    scenes = range(num_of_scenes)

    # make figure
    figure = {
        'data': [],
        'layout': {},
        'frames': []
    }

    # fill in most of layout
    figure['layout']['xaxis'] = {'range': [-120, 120], 'title': 'x-axis'}
    figure['layout']['yaxis'] = {'title': 'y-axis'}
    figure['layout']['hovermode'] = 'closest'
    figure['layout']['sliders'] = {
        'args': [
            'transition', {
                'duration': 400,
                'easing': 'cubic-in-out'
            }
        ],
        'initialValue': '0',
        'plotlycommand': 'animate',
        'values': scenes,
        'visible': True
    }
    figure['layout']['updatemenus'] = [
        {
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 500, 'redraw': False},
                             'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                    'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }
    ]

    sliders_dict = {
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Scene:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': []
    }

    # make data
    scene = 0

    object_by_scene = objects[scene]
    x = tuple(object_by_scene['X'].tolist())
    y = tuple(object_by_scene['Y'].tolist())
    z = tuple(object_by_scene['Z'].tolist())
    radius = tuple(object_by_scene['radius'].tolist())
    if view==1:
        x_points = x
        y_points = z
    
    elif view==2:
        x_points = x
        y_points = y

    elif view==3:
        x_points = z
        y_points = y
    
    else:
        print("Not a valid view")
        x_points = x
        y_points = z
        
    data_dict = {
        'x': x_points,
        'y': y_points,
        'mode': 'markers',
        'marker': {
                'color':radius,
                'size': 2,
                'opacity':1        
        }
    }
    figure['data'].append(data_dict)

    
    # make frames
    for scene in scenes:
        frame = {'data': [], 'name': str(scene)}

        object_by_scene = objects[scene]
        x = tuple(object_by_scene['X'].tolist())
        y = tuple(object_by_scene['Y'].tolist())
        z = tuple(object_by_scene['Z'].tolist())
        radius = tuple(object_by_scene['radius'].tolist())
        if view==1:
            x_points = x
            y_points = z
        elif view==2:
            x_points = x
            y_points = y
        elif view==3:
            x_points = z
            y_points = y
        else:
            print("Not a valid view")
            x_points = x
            y_points = z

        data_dict = {
            'x': x_points,
            'y': y_points,
            'mode': 'markers',
            'marker': {
                    'color':radius,
                    'size': 2,
                    'opacity':1        
            }
        }
        frame['data'].append(data_dict)

        figure['frames'].append(frame)
        
        slider_step = {'args': [
            [scene],
            {'frame': {'duration': 300, 'redraw': False},
             'mode': 'immediate',
           'transition': {'duration': 300}}
         ],
         'label': scene,
         'method': 'animate'}
        sliders_dict['steps'].append(slider_step)


    figure['layout']['sliders'] = [sliders_dict]

    plot(figure, filename = "../visuals/{}_{}_{}.html".format(object_name, num_of_scenes, view),auto_open=False, show_link = False)

## Creating the list of scenes

In [9]:
dataframes = read_file(file_path)

## removing the outliers and creating the list of object points

In [10]:
objects = remove_outliers(dataframes, num_of_scenes, outliers_path )

## Plotting the object points in the desire view

View | Number
---------- | -----------
XZ | 1
XY | 2
YZ | 3

In [11]:
# Select only the desired scene and desired angle
scene_to_plot = objects[34]
view = 3

# plotting the object with desired view
visualize(scene_to_plot, view)

In [12]:
# Creating the animation 
# The animation is created as a html file at src/visual/objectName_{num_of_scenes}_{view}
visualize_50_html(num_of_scenes=50,view=3)

## Fix the size of the images and 